# Neural Network from Scratch
Code modified from https://github.com/adventuresinML/adventures-in-ml-code/blob/master/neural_network_tutorial.py

The notation in this website is almost the same as the notation we are using in class.  Instead of $a$ the author uses $h$, and instead of $N$, the author uses $m$. (I have modified the code below to use $a$ and $N$.)

Please read about this implementation starting at page 27 from the website listed above.

## The first thing we will do is import all the libraries

We will be using the lower resolution MINST data set

In [1]:
from sklearn.datasets import load_digits # The MNIST data set is in scikit learn data set
from sklearn.preprocessing import StandardScaler  # It is important in neural networks to scale the date
from sklearn.model_selection import train_test_split  # The standard - train/test to prevent overfitting and choose hyperparameters
from sklearn.metrics import accuracy_score # 
import numpy as np
import numpy.random as r # We will randomly initialize our weights
import matplotlib.pyplot as plt 

import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import copy
from itertools import permutations


## Looking at the data

After we load the data, we print the shape of the data and a pixelated digit.

We also show what the features of one example looks like.

The neural net will learn to estimate which digit these pixels represent.

In [2]:
# After completing the code in this code cell, run this code cell before moving further.
names =[
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 
    'restecg',  'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'
]

# TODO - write the pandas command to read the csv file into a dataframe df - 5 points
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data',
                 header = None, names = names, na_values = '?')

df = df.dropna()
# print(df.shape)
# print(df.shape)

x=np.array(df.iloc[:,0:13])
y=np.array(df.iloc[:,13:14])

#The goal is to distinguish the presence from non-presense 
for i in range(len(y)):
    if(y[i]>0):
        y[i] = 1
#print(y)

# print(x.shape)
# print(y.shape)

## 1) Scale the dataset
The training features range from 0 to 15.  To help the algorithm converge, we will scale the data to have a mean of 0 and unit variance

In [3]:
X_scale = StandardScaler()
X = X_scale.fit_transform(x)

# X_orig = copy.deepcopy(X)
# y_orig = copy.deepcopy(y)

X[0,:] # Looking the new features after scaling
#print(X.shape[1])

array([ 0.93618065,  0.69109474, -2.24062879,  0.75038004, -0.27644339,
        2.43042728,  1.01019851,  0.01749443, -0.69641941,  1.06896529,
        2.26414539, -0.72197605,  0.65587737])

## 2) Creating training and test datasets
We split the data into training and test data sets. We will train the neural network with the training dataset, and evaluate our neural network with the test dataset 

In [4]:
#Split the data into training and test set.  60% training and %40 test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
#print(y_test)

## 3) Setting up the output layer

### One hot encoding
Our target is an integer in the range [0,..,9], so we will have 10 output neuron's in our network.  

-  If  $y=0$, we want the output neurons to have the values $(1,0,0,0,0,0,0,0,0,0)$

-  If  $y=1$ we want the output neurons to have the values $(0,1,0,0,0,0,0,0,0,0)$
-  etc

Thus we need to change our target so it is the same as our hoped for output of the neural network.  
-  If $y=0$ we change it into the vector $(1,0,0,0,0,0,0,0,0,0)$. 
-  If $y=1$ we change it into the vector $(0,1,0,0,0,0,0,0,0,0)$
-  etc

See page 29 from the website listed above

The code to covert the target vector.

In [5]:
# def convert_y_to_vect(y):
#     y_vect = np.zeros((len(y), 10))
#     for i in range(len(y)):
#         y_vect[i, y[i]] = 1
#     return y_vect

Converting the training and test targets to vectors 

In [6]:
# convert digits to vectors
# y_v_train = convert_y_to_vect(y_train)
# y_v_test = convert_y_to_vect(y_test)

A quick check to see that our code performs as we expect 

In [7]:
# print(y_train[0:4])
# print(y_v_train[0:4])

## 4) Creating the neural network

### The activation function and its derivative

We will use the sigmoid activation function:  $f(z)=\frac{1}{1+e^{-z}}$

The deriviative of the sigmoid function is: $f'(z) = f(z)(1-f(z))$ 

In [8]:
def f(z):
    return 1 / (1 + np.exp(-z))


def f_deriv(z):
    return f(z) * (1 - f(z))

### Creating and initialing W and b
We want the weights in W to be different so that during back propagation the nodes on a level will have different gradients and thus have different update values.

We want the  weights to be small values, since the sigmoid is almost "flat" for large inputs.

Next is the code that assigns each weight a number uniformly drawn from $[0.0, 1.0)$.  The code assumes that the number of neurons in each level is in the python list *nn_structure*.

In the code, the weights, $W^{(\ell)}$ and $b^{(\ell)}$ are held in a python dictionary

In [9]:
def setup_and_init_weights(nn_structure):
    W = {} #creating a dictionary i.e. a set of key: value pairs
    b = {}
    for l in range(1, len(nn_structure)):
        W[l] = r.random_sample((nn_structure[l], nn_structure[l-1])) #Return “continuous uniform” random floats in the half-open interval [0.0, 1.0). 
        b[l] = r.random_sample((nn_structure[l],))
    return W, b

### Initializing $\triangledown W$ and $\triangledown b$
Creating $\triangledown W^{(\ell)}$ and $\triangledown b^{(\ell)}$ to have the same size as $W^{(\ell)}$ and $b^{(\ell)}$, and setting $\triangledown W^{(\ell)}$, and  $\triangledown b^{(\ell)}$ to zero

In [10]:
def init_tri_values(nn_structure):
    tri_W = {}
    tri_b = {}
    for l in range(1, len(nn_structure)):
        tri_W[l] = np.zeros((nn_structure[l], nn_structure[l-1]))
        tri_b[l] = np.zeros((nn_structure[l],))
    return tri_W, tri_b

## Feed forward
Perform a forward pass throught the network.  The function returns the values of $a$ and $z$

In [11]:
def feed_forward(x, W, b):
    a = {1: x} # create a dictionary for holding the a values for all levels
    z = { } # create a dictionary for holding the z values for all the layers
    for l in range(1, len(W) + 1): # for each layer
        node_in = a[l]
        z[l+1] = W[l].dot(node_in) + b[l]  # z^(l+1) = W^(l)*a^(l) + b^(l)
        a[l+1] = f(z[l+1]) # a^(l+1) = f(z^(l+1))
    return a, z

## Compute $\delta$
The code below compute $\delta^{(s_l)}$ in a function called "calculate_out_layer_delta",  and  computes $\delta^{(\ell)}$ for the hidden layers in the function called "calculate_hidden_delta".  

If we wanted to have a different cost function, we would change the "calculate_out_layer_delta" function.


In [12]:
def calculate_out_layer_delta(y, a_out, z_out):
    # delta^(nl) = -(y_i - a_i^(nl)) * f'(z_i^(nl))
    return -(y-a_out) * f_deriv(z_out) 


def calculate_hidden_delta(delta_plus_1, w_l, z_l):
    # delta^(l) = (transpose(W^(l)) * delta^(l+1)) * f'(z^(l))
    return np.dot(np.transpose(w_l), delta_plus_1) * f_deriv(z_l)

## The Back Propagation Algorithm


In [13]:
# l=1, input layer

def train_nn(nn_structure, X, y, iter_num=3000, alpha=0.5, lamb=0):
    W, b = setup_and_init_weights(nn_structure)
    cnt = 0
    N = len(y)
    avg_cost_func = []
    print('Starting gradient descent for {} iterations'.format(iter_num))
    
    
    while cnt < iter_num:
        if cnt%1000 == 0:
            print('Iteration {} of {}'.format(cnt, iter_num))
        # clean derivative of Ws, since its different every time    
        tri_W, tri_b = init_tri_values(nn_structure)
        
        avg_cost = 0
        
        # for every training sample set
        
        for i in range(N):
            delta = {}
            # perform the feed forward pass and return the stored a and z values, to be used in the
            # gradient descent step
            a, z = feed_forward(X[i, :], W, b)
            # loop from nl-1 to 1 backpropagating the errors
            for l in range(len(nn_structure), 0, -1):
                # last/output layer
                if l == len(nn_structure):
                    delta[l] = calculate_out_layer_delta(y[i,:], a[l], z[l])
                    avg_cost += np.linalg.norm((y[i,:]-a[l]))
                else:
                    #Remember that J/W_l = delta[l+1!!!]*a[l]
                    if l > 1:
                        delta[l] = calculate_hidden_delta(delta[l+1], W[l], z[l])
                    # Since tri W_l = sum(J/w^l) for all xs --> J/w = delta_l+1 * a_l     
                    # triW^(l) = triW^(l) + delta^(l+1) * transpose(a^(l))
                    tri_W[l] += np.dot(delta[l+1][:,np.newaxis], np.transpose(a[l][:,np.newaxis]))# np.newaxis increase the number of dimensions
                    # trib^(l) = trib^(l) + delta^(l+1)
                    tri_b[l] += delta[l+1]
                    
        # this entire for i in N loop above is updating the  deri. of all W_ls
                    
        # perform the gradient descent step for the weights in EACH LAYER! 
        for l in range(len(nn_structure) - 1, 0, -1):
            # wl and tri_W[l] are both 1D array/vectors
            W[l] += -alpha * (1.0/N * tri_W[l] + lamb*W[l])
            b[l] += -alpha * (1.0/N * tri_b[l])
        # complete the average cost calculation
        avg_cost = 1.0/N * avg_cost
        avg_cost_func.append(avg_cost)
        cnt += 1
    #print(avg_cost_func)
    return W, b, avg_cost_func


def predict_y(W, b, X, n_layers, threshold):
    N = X.shape[0]
    y = np.zeros((N,))
    
    for i in range(N):
        a, z = feed_forward(X[i, :], W, b)
        if(a[n_layers] > threshold):
            y[i] = 1
        else:
            y[i] = 0         
    return y

def predict_yhat(W, b, X, n_layers, threshold):
    N = X.shape[0]
    y = np.zeros((N,))
    
    for i in range(N):
        a, z = feed_forward(X[i, :], W, b)
        if(a[n_layers] > threshold):
            y[i] = 1
        else:
            y[i] = 0         
    return y



## Running the neural network

Our code assumes the size of each layer in our network is held in a list.  The input layer will have 64 neurons (one for each pixel in our 8 by 8 pixelated digit).  Our hidden layer has 30 neurons (you can change this value).  The output layer has 10 neurons.

Next we create the python list to hold the number of neurons for each level and then run the neural network code with our training data.

This code will take some time...

In [14]:
#input layer, 1 hidden layer, 1 output


# train the NN

acc_300 = []
acc_600 = []

def main(n_neuro, lambs, num_iter=3000):
    acc_test = []
    acc_train = []
    nn_structure = [13, 1]
    nn_structure.insert(1,n_neuro)
    
    for i in range(len(lambs)):
        W, b, avg_cost_func = train_nn(nn_structure, X_train, y_train, num_iter, lambs[i])

#         plt.plot(avg_cost_func)
#         plt.ylabel('Average J')
#         plt.xlabel('Iteration number')
#         plt.show()
        y_hat =  predict_yhat(W, b, X_train, 3, 0.5)
        y_pred = predict_y(W, b, X_test, 3, 0.5)
        
        print('Training accuracy is {}%'.format(accuracy_score(y_train, y_hat) * 100))
        print('Prediction accuracy is {}%'.format(accuracy_score(y_test, y_pred) * 100))
        
        acc_train.append([lambs[i],accuracy_score(y_train, y_hat)])
        acc_test.append([lambs[i],accuracy_score(y_test, y_pred)])
    
    return acc_train, acc_test

In [17]:
res_4_small = []
lambs_4_small = np.geomspace(0.001, 3, 30)
#print(lambs_4_small)
tr_4_002, test_4_002 = main(4, lambs_4_small, 3000)
print(tr_4_002, test_4_002)

Starting gradient descent for 3000 iterations
Iteration 0 of 3000
Iteration 1000 of 3000
Iteration 2000 of 3000
Training accuracy is 43.82022471910113%
Prediction accuracy is 49.57983193277311%
Starting gradient descent for 3000 iterations
Iteration 0 of 3000
Iteration 1000 of 3000
Iteration 2000 of 3000
Training accuracy is 50.56179775280899%
Prediction accuracy is 58.82352941176471%
Starting gradient descent for 3000 iterations
Iteration 0 of 3000
Iteration 1000 of 3000
Iteration 2000 of 3000
Training accuracy is 58.42696629213483%
Prediction accuracy is 69.74789915966386%
Starting gradient descent for 3000 iterations
Iteration 0 of 3000
Iteration 1000 of 3000
Iteration 2000 of 3000
Training accuracy is 43.82022471910113%
Prediction accuracy is 50.42016806722689%
Starting gradient descent for 3000 iterations
Iteration 0 of 3000
Iteration 1000 of 3000
Iteration 2000 of 3000
Training accuracy is 66.29213483146067%
Prediction accuracy is 70.58823529411765%
Starting gradient descent for 

In [20]:
for elem in tr_4_002:
    print(elem)
print()
for elem in test_4_002:
    print(elem)

[0.001, 0.43820224719101125]
[0.0013179554578665073, 0.5056179775280899]
[0.0017370065889201168, 0.5842696629213483]
[0.0022892973142173523, 0.43820224719101125]
[0.0030171918899518993, 0.6629213483146067]
[0.003976524518792668, 0.6460674157303371]
[0.005240882192882783, 0.702247191011236]
[0.006907249290145261, 0.7247191011235955]
[0.009103446900791505, 0.7191011235955056]
[0.01199793752829611, 0.7584269662921348]
[0.01581274724855926, 0.7528089887640449]
[0.020840496540102282, 0.7808988764044944]
[0.02746684615967588, 0.7752808988764045]
[0.03620007980652456, 0.7528089887640449]
[0.047710092756212186, 0.7865168539325843]
[0.06287977714336719, 0.7865168539325843]
[0.08287274547553049, 0.8314606741573034]
[0.10922258720785737, 0.8539325842696629]
[0.14395050493289618, 0.8651685393258427]
[0.18972035363895032, 0.8764044943820225]
[0.2500429755468184, 0.8932584269662921]
[0.3295455043231113, 0.8932584269662921]
[0.43432629603801526, 0.8876404494382022]
[0.5724227123582466, 0.910112359550